## Computational Chemistry for Experimentalists
## Module 12: NMR

NMR spectroscopy is central to organic chemistry. Here we predict NMR shieldings and spin-spin couplings to predict full NMR spectra. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

from pyscf.tools import cubegen 
import py3Dmol
import numpy
import tabulate
from pyscf import gto,scf,tdscf,dft,tddft
from pyscf.tools import cubegen 
from pyscf.prop import nmr,ssc
import matplotlib.pyplot as plt

In [ ]:
def RDKitToPySCF(rdkitmol,basis='3-21g'):
    elements = [atom.GetSymbol() for atom in rdkitmol.GetAtoms()]
    coordinates = rdkitmol.GetConformer().GetPositions()
    atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

    pyscf_mole = gto.Mole(basis=basis)
    pyscf_mole.atom = atoms
    pyscf_mole.build();
    return(pyscf_mole)
    

## Part 1: Acetaldehyde proton NMR 

NMR measures the energy levels of a nuclear spin aligned with an external static magnetic field. Normal DFT calculations assume all nuclear spins and external magnetic fields are zero. We compute the NMR shielding as the second derivative of energy with respect to nuclear spin and magnetic field. These should be computed relative to a reference molecule, typically TMS

This first block computes RDKit geometries for acetaldehyde and TMS, converts them to PySCF molecules, and visualizes them 

In [ ]:
m0=Chem.MolFromSmiles('C[Si](C)(C)C')
m1=Chem.AddHs(m0)
AllChem.EmbedMolecule(m1)
AllChem.MMFFOptimizeMolecule(m1)
mb1=Chem.MolToMolBlock(m1)
mm1=RDKitToPySCF(m1)

m0=Chem.MolFromSmiles('O=CC')
m2=Chem.AddHs(m0)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
mb2=Chem.MolToMolBlock(m2)
mm2=RDKitToPySCF(m2)

p=py3Dmol.view(width=600,height=400,viewergrid=(1,2))
p.addModel(mb1,'sdf',viewer=(0,0))
p.addModel(mb2,'sdf',viewer=(0,1))
p.addPropertyLabels("index","",{'showBackground':'false','alignment': 'center'})
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

The next block does HF/3-21G calculations at these geometries, and computes absolute NMR shieldings of all nuclei. Arrays sh1 and sh2 contain a Natomx3x3 array with a 3x3  shielding tensor (for magnetic fields and nuclear magnetic dipole moments in x,y,z directions) for each nucleus. In solution, the molecule tumbles relative to the applied field, so we average the x,x y,y and z,z shieldings. 

In [ ]:
mf1=scf.RHF(mm1)
mf1.kernel()
mn1=nmr.rhf.NMR(mf1)
sh1=mn1.kernel()
mf2=scf.RHF(mm2)
mf2.kernel()
mn2=nmr.rhf.NMR(mf2)
sh2=mn2.kernel()

In [ ]:
#First we determine the average magnetic shielding of all TMS protons
numH=0
sumShield=0
for i in range(mm1.natm):
    if(mm1.atom_charge(i)==1):
        numH=numH+1
        val=(sh1[i,0,0]+sh1[i,1,1]+sh1[i,2,2])/3
        sumShield=sumShield+val
        #print('%2d %2d %6.2f'%(i,mm1.atom_charge(i),val))
aveShield=sumShield/numH
print('TMS proton absolute isotropic shielding:',aveShield)

In [ ]:
# Next we determine the chemical shifts of each acetaldehyde proton. 
print('Shielding of each acetaldehyde proton:\nIndex  Shielding (ppm)')
for i in range(mm2.natm):
    if(mm2.atom_charge(i)==1):
        val=(sh2[i,0,0]+sh2[i,1,1]+sh2[i,2,2])/3
        print('%2d %6.2f %7.3f '%(i,aveShield-val,val))

The experimental acetaldehyde NMR has a peak around 2.2 ppm integrated intensity 3x, and a peak around 9.8 ppm integrated intensity 1x. Based on the picture above, atoms 4-6 are the methyl protons and atom 3 is the aldehyde proton. Recall from Module 4 that the methyl proton shifts are rotationally averaged at normal temperatures, where methyl rotation is fast on the NMR timescale. 

## Part 2: Spin-Spin Coupling 

Hyperfine interactions spin-spin coupling are useful for structure assignment. Here we compute the 13C-1H spin-spin coupling of isotopically labeled methane. Experimentally, the methane proton at 0.23 ppm is split into a doublet with width 125 Hz, see https://pubs.acs.org/doi/10.1021/ct050083f . The splitting  is independent of the external magnetic field strength, thus different NMR spectrometers will give  different looking spectra. For more complicated molecules, spin-spin couplings must be thermally averaged. 

In [ ]:
m0=Chem.MolFromSmiles('C')
m3=Chem.AddHs(m0)
AllChem.EmbedMolecule(m3)
AllChem.MMFFOptimizeMolecule(m3)
mb3=Chem.MolToMolBlock(m3)
mm3=RDKitToPySCF(m3)
p=py3Dmol.view(width=200,height=200)
p.addModel(mb3,'sdf')
p.addPropertyLabels("index","",{'showBackground':'false','alignment': 'center'})
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

In [ ]:
mf3=scf.RHF(mm3)
mf3.kernel()
mn3=nmr.rhf.NMR(mf3)
sh3=mn3.kernel()

In [ ]:
print('Shielding of each methane proton:\nIndex  Shielding (ppm)')
for i in range(mm3.natm):
    if(mm3.atom_charge(i)==1):
        val=(sh3[i,0,0]+sh3[i,1,1]+sh3[i,2,2])/3
        print('%2d %6.2f '%(i,aveShield-val))

In [ ]:
ms3=ssc.RHF(mf3)
ss3=ms3.kernel()

The 13C-1H spin-spin coupling is computed as 119 Hz, close to the 125 Hz seen experimentally. 

In [ ]:
m=gto.Mole(atom='H',charge=0,spin=1,basis='3-21g')
m.build()
mf=scf.UHF(m)
mf.kernel()
mn=nmr.uhf.NMR(mf)
sh=mn.kernel()


## Part 3: Method and Basis Set 

NMR shifts are sensitive to details of method and basis set. You should try on your own to compute the methane chemical shift and 13C-1H SSCC with different DFT methods and basis sets and geometry optimization. 

Your homework for this module is to compute the  B3LYP/6-31G(d) proton NMR spectra of 3-Dimethylaminoacrolein at low and high temperatures. You'll need to determine which protons to rotationally average. You should compare your results to the experiments at http://www1.chem.umn.edu/groups/taton/chem8361/Handouts/9_26.pdf . 

You should also compare the rotationally averaged total shielding of H(+) vs. H2 vs. H(-), to confirm that the electrons shield the nucleus and reduce the magnitude of the derivative coupling. 

For 50 points extra credit, report the predicted 1H NMR spectrum of 13C-labeled methane as measured with the TCU NMR, including both the chemical shifts and the spin-spin couplings. 
